# 敵対的生成ネットワーク（GAN、Generative Adversarial Nets）
ここでは、Kerasを用いて、GANの学習を行う。  
GANによって生成された画像は、imagesフォルダに保存される。  

In [1]:
%matplotlib inline
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam

import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


### MNISTデータの読み込み

In [2]:
# Load the MNIST dataset
import tensorflow as tf
mnist = tf.keras.datasets.mnist
(X_train, y_train),(X_test, y_test) = mnist.load_data()

# 28*28の画像データを784のベクトルに変換する
X_train = X_train.reshape(-1, 784) / 255
X_test = X_test.reshape(-1, 784) / 255

# N, H,W,Cの配列形状に変換する
X_train = X_train.reshape(-1, 28, 28, 1)

# -1から1の範囲に収める.  generatorの出力にtanhを設定するため
X_train = (X_train * 2) -1

### 条件設定

In [3]:
# 画像サイズ
img_rows = 28
img_cols = 28
# 画像のチャンネル数
channels = 1
# 画像の形式
img_shape = (img_rows, img_cols, channels)

### optimizerを設定

In [4]:
optimizer = Adam(0.0002, 0.5)

### Generatorの定義

In [5]:
def build_generator():
    
    # 入力(ノイズが入いる)
    noise_shape = (100,)

    model = Sequential()

    # 全結合層
    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # 全結合層
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # 全結合層
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # 全結合層
    model.add(Dense(np.prod(img_shape), activation='tanh')) # np.prod((1,2,3))=6
    model.add(Reshape(img_shape))

    # ネットワーク構成の表示
    model.summary()

    # 入力レイヤを定義
    noise = Input(shape=noise_shape)
    
    # modelの出力を定義
    img = model(noise)

    # モデルの定義
    # ノイズを入力し, 画像データを出力するモデルを定義
    return Model(inputs=noise, outputs=img) 

### Discriminatorの定義

In [6]:
def build_discriminator():

    model = Sequential()

    # 画像形式配列を1次元配列に変換
    model.add(Flatten(input_shape=img_shape))
    
    # 全結合層
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    
    # 全結合層
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    
    # 全結合層
    model.add(Dense(1, activation='sigmoid'))
    
    # ネットワーク構成の表示
    model.summary()
    
    # 入力レイヤを定義
    img = Input(shape=img_shape)
    
    # modelの出力を定義. probは0~1の値
    prob = model(img)
    
    # モデルの定義
    # 画像データを入力すると, 0~1の値を返すモデルを定義
    return Model(inputs=img, outputs=prob) 

### Discriminatorの構築とコンパイル

In [7]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# ラベルは0or1なので、binary_crossentropyは、画像ごとに2値分類時のクロスエントロピー誤差を算出している




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________



### GeneratorとDiscriminatorを結合したモデルの構築とコンパイル

In [8]:
# Generatorの構築
generator = build_generator()

# Generatorはノイズを入力として画像を生成する
z = Input(shape=(100,))
img_to_d = generator(z)

# 結合したモデルでは、Generatorしか訓練しないので、パラメータを更新しないように設定しておく
discriminator.trainable = False

# モデルの結合: 入力zにはノイズが入る。 discriminator(img_to_d)は識別器での識別結果
combined = Model(z, discriminator(img_to_d))
combined.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
# ラベルは0or1なので、binary_crossentropyは、画像ごとに2値分類時のクロスエントロピー誤差を算出している

# ネットワーク構成の表示
combined.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_5 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              525312    
_________

### 画像を生成し保存する関数を定義

In [9]:
def sample_images(epoch):
    # 画像の枚数
    r, c = 5, 5
    
    # サンプリング
    noise = np.random.normal(loc=0, scale=1, size=(r * c, 100)) # 画像1枚につき、100点をサンプリングする
    
    # 画像の生成(gen_imgsは-1~1の値)
    gen_imgs = generator.predict(noise)

    # 0から1の範囲に収める
    gen_imgs = 0.5 * gen_imgs + 0.5

    # 5*5枚の画像を並べて1枚の画像にする
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
            
    # 画像を出力
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

### 訓練
訓練しながら、生成された画像を一定間隔で保存する。

In [10]:
epochs = 10000
batch_size = 32
sample_interval = 200 # 画像保存間隔
half_batch_size = int(batch_size / 2) # Discriminatorの学習では、バッチサイズの半分を正解データにし残り半分を偽物データにする


for epoch in range(epochs):

    # ランダムにhalf_batch_sizeの分だけ画像を選択
    idx = np.random.randint(0, X_train.shape[0], half_batch_size)
    imgs = X_train[idx]

    # ノイズをサンプリング
    noise = np.random.normal(0, 1, (half_batch_size, 100))

    # half_batch_sizeの分だけ画像を生成
    gen_imgs = generator.predict(noise)

    # 正解データが半分、偽物データが半分のデータセットをつくる
    concat_data = np.concatenate([imgs, gen_imgs], axis=0)
    concat_label = np.concatenate([np.ones((half_batch_size, 1)), np.zeros((half_batch_size, 1))], axis=0)
    
    # Discriminatorを訓練    
    d_loss, d_acc = discriminator.train_on_batch(concat_data, concat_label)

    
    #  Generatorを訓練するのに必要なノイズを生成する
    noise = np.random.normal(loc=0, scale=1, size=(batch_size, 100))

    # ラベル=1を設定する. GeneratorはDiscriminatorに生成した画像を本物と判定させたい
    label = np.ones((batch_size, 1))

    # 結合したモデルを使ってGeneratorを訓練する
    g_loss, g_acc = combined.train_on_batch(noise, label) # ノイズ, ラベル=1

    # 進捗を表示
    print ("%d [d_loss: %f, d_acc:%s] [g_loss: %f, g_acc:%s]" % (epoch, d_loss, d_acc, g_loss, g_acc))

    # 一定間隔で画像を保存
    if epoch % sample_interval == 0:
        sample_images(epoch)

/Users/ogata/.pyenv/versions/3.7.3/envs/DLvr6/lib/python3.7/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [d_loss: 0.449199, d_acc:0.5625] [g_loss: 0.411263, g_acc:0.96875]


/Users/ogata/.pyenv/versions/3.7.3/envs/DLvr6/lib/python3.7/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [d_loss: 0.642750, d_acc:0.5] [g_loss: 0.500427, g_acc:0.875]
2 [d_loss: 0.540718, d_acc:0.53125] [g_loss: 0.587158, g_acc:0.78125]
3 [d_loss: 0.436096, d_acc:0.59375] [g_loss: 0.715122, g_acc:0.53125]
4 [d_loss: 0.393586, d_acc:0.75] [g_loss: 0.800269, g_acc:0.40625]
5 [d_loss: 0.329535, d_acc:0.84375] [g_loss: 0.951207, g_acc:0.125]
6 [d_loss: 0.285353, d_acc:0.875] [g_loss: 1.072358, g_acc:0.09375]
7 [d_loss: 0.286281, d_acc:0.90625] [g_loss: 1.238253, g_acc:0.03125]
8 [d_loss: 0.192131, d_acc:1.0] [g_loss: 1.279840, g_acc:0.03125]
9 [d_loss: 0.213508, d_acc:1.0] [g_loss: 1.446612, g_acc:0.0625]
10 [d_loss: 0.155513, d_acc:1.0] [g_loss: 1.545521, g_acc:0.0]
11 [d_loss: 0.144336, d_acc:1.0] [g_loss: 1.632294, g_acc:0.0]
12 [d_loss: 0.146161, d_acc:1.0] [g_loss: 1.762371, g_acc:0.0]
13 [d_loss: 0.161848, d_acc:1.0] [g_loss: 1.795204, g_acc:0.0]
14 [d_loss: 0.112912, d_acc:1.0] [g_loss: 1.862191, g_acc:0.0]
15 [d_loss: 0.103802, d_acc:1.0] [g_loss: 1.875254, g_acc:0.0]
16 [d_loss: 0.

133 [d_loss: 0.012032, d_acc:1.0] [g_loss: 4.784137, g_acc:0.0]
134 [d_loss: 0.027564, d_acc:1.0] [g_loss: 4.678493, g_acc:0.0]
135 [d_loss: 0.007424, d_acc:1.0] [g_loss: 4.753983, g_acc:0.0]
136 [d_loss: 0.009720, d_acc:1.0] [g_loss: 4.675068, g_acc:0.0]
137 [d_loss: 0.020776, d_acc:1.0] [g_loss: 4.905867, g_acc:0.0]
138 [d_loss: 0.027341, d_acc:1.0] [g_loss: 5.150681, g_acc:0.0]
139 [d_loss: 0.084331, d_acc:0.96875] [g_loss: 4.004547, g_acc:0.0625]
140 [d_loss: 0.028539, d_acc:1.0] [g_loss: 3.636714, g_acc:0.0625]
141 [d_loss: 0.106349, d_acc:0.9375] [g_loss: 4.547465, g_acc:0.0]
142 [d_loss: 0.015573, d_acc:1.0] [g_loss: 5.033241, g_acc:0.0]
143 [d_loss: 0.052638, d_acc:1.0] [g_loss: 4.927732, g_acc:0.0]
144 [d_loss: 0.030242, d_acc:1.0] [g_loss: 4.575390, g_acc:0.03125]
145 [d_loss: 0.042208, d_acc:1.0] [g_loss: 5.121633, g_acc:0.0]
146 [d_loss: 0.032673, d_acc:1.0] [g_loss: 5.038945, g_acc:0.0]
147 [d_loss: 0.035888, d_acc:1.0] [g_loss: 4.169943, g_acc:0.03125]
148 [d_loss: 0.0589

...

9509 [d_loss: 0.754959, d_acc:0.46875] [g_loss: 0.681889, g_acc:0.65625]
9510 [d_loss: 0.696945, d_acc:0.53125] [g_loss: 0.717595, g_acc:0.5]
9511 [d_loss: 0.695832, d_acc:0.53125] [g_loss: 0.794816, g_acc:0.375]
9512 [d_loss: 0.659465, d_acc:0.59375] [g_loss: 0.812873, g_acc:0.28125]
9513 [d_loss: 0.614933, d_acc:0.6875] [g_loss: 0.862234, g_acc:0.21875]
9514 [d_loss: 0.692738, d_acc:0.53125] [g_loss: 0.829132, g_acc:0.21875]
9515 [d_loss: 0.622766, d_acc:0.65625] [g_loss: 0.785698, g_acc:0.40625]
9516 [d_loss: 0.632244, d_acc:0.625] [g_loss: 0.738536, g_acc:0.4375]
9517 [d_loss: 0.686895, d_acc:0.53125] [g_loss: 0.769779, g_acc:0.4375]
9518 [d_loss: 0.593036, d_acc:0.75] [g_loss: 0.751030, g_acc:0.4375]
9519 [d_loss: 0.565427, d_acc:0.8125] [g_loss: 0.884066, g_acc:0.3125]
9520 [d_loss: 0.695500, d_acc:0.59375] [g_loss: 0.763127, g_acc:0.5]
9521 [d_loss: 0.678240, d_acc:0.5625] [g_loss: 0.862617, g_acc:0.28125]
9522 [d_loss: 0.633094, d_acc:0.59375] [g_loss: 0.708547, g_acc:0.53125]


9630 [d_loss: 0.646329, d_acc:0.65625] [g_loss: 0.754626, g_acc:0.4375]
9631 [d_loss: 0.616218, d_acc:0.65625] [g_loss: 0.852305, g_acc:0.3125]
9632 [d_loss: 0.702055, d_acc:0.53125] [g_loss: 0.782864, g_acc:0.375]
9633 [d_loss: 0.683323, d_acc:0.46875] [g_loss: 0.575446, g_acc:0.625]
9634 [d_loss: 0.675441, d_acc:0.59375] [g_loss: 0.709165, g_acc:0.46875]
9635 [d_loss: 0.608244, d_acc:0.625] [g_loss: 0.689210, g_acc:0.5625]
9636 [d_loss: 0.700111, d_acc:0.59375] [g_loss: 0.783790, g_acc:0.28125]
9637 [d_loss: 0.670031, d_acc:0.625] [g_loss: 0.809536, g_acc:0.375]
9638 [d_loss: 0.664207, d_acc:0.625] [g_loss: 0.857707, g_acc:0.3125]
9639 [d_loss: 0.754314, d_acc:0.4375] [g_loss: 0.797626, g_acc:0.34375]
9640 [d_loss: 0.684015, d_acc:0.59375] [g_loss: 0.684260, g_acc:0.46875]
9641 [d_loss: 0.655812, d_acc:0.65625] [g_loss: 0.702668, g_acc:0.5625]
9642 [d_loss: 0.660475, d_acc:0.59375] [g_loss: 0.869815, g_acc:0.28125]
9643 [d_loss: 0.663742, d_acc:0.5625] [g_loss: 0.831796, g_acc:0.3437

9753 [d_loss: 0.666004, d_acc:0.625] [g_loss: 0.786007, g_acc:0.40625]
9754 [d_loss: 0.721186, d_acc:0.59375] [g_loss: 0.740771, g_acc:0.40625]
9755 [d_loss: 0.599816, d_acc:0.65625] [g_loss: 0.799278, g_acc:0.40625]
9756 [d_loss: 0.679520, d_acc:0.6875] [g_loss: 0.848788, g_acc:0.3125]
9757 [d_loss: 0.661591, d_acc:0.59375] [g_loss: 0.880749, g_acc:0.3125]
9758 [d_loss: 0.683293, d_acc:0.6875] [g_loss: 0.788408, g_acc:0.3125]
9759 [d_loss: 0.700807, d_acc:0.53125] [g_loss: 0.742880, g_acc:0.53125]
9760 [d_loss: 0.737688, d_acc:0.46875] [g_loss: 0.680882, g_acc:0.59375]
9761 [d_loss: 0.693256, d_acc:0.59375] [g_loss: 0.745573, g_acc:0.46875]
9762 [d_loss: 0.661124, d_acc:0.625] [g_loss: 0.638309, g_acc:0.59375]
9763 [d_loss: 0.626580, d_acc:0.59375] [g_loss: 0.788267, g_acc:0.28125]
9764 [d_loss: 0.665957, d_acc:0.59375] [g_loss: 0.800511, g_acc:0.34375]
9765 [d_loss: 0.725533, d_acc:0.5] [g_loss: 0.703616, g_acc:0.46875]
9766 [d_loss: 0.659392, d_acc:0.5] [g_loss: 0.715552, g_acc:0.56

9872 [d_loss: 0.712267, d_acc:0.53125] [g_loss: 0.907652, g_acc:0.25]
9873 [d_loss: 0.644575, d_acc:0.5625] [g_loss: 0.967636, g_acc:0.21875]
9874 [d_loss: 0.709156, d_acc:0.5625] [g_loss: 0.834869, g_acc:0.4375]
9875 [d_loss: 0.616399, d_acc:0.65625] [g_loss: 0.699111, g_acc:0.5625]
9876 [d_loss: 0.612570, d_acc:0.65625] [g_loss: 0.711700, g_acc:0.5625]
9877 [d_loss: 0.694536, d_acc:0.5625] [g_loss: 0.839300, g_acc:0.4375]
9878 [d_loss: 0.704772, d_acc:0.5625] [g_loss: 0.917546, g_acc:0.28125]
9879 [d_loss: 0.677181, d_acc:0.5625] [g_loss: 1.004531, g_acc:0.1875]
9880 [d_loss: 0.669933, d_acc:0.6875] [g_loss: 0.828694, g_acc:0.34375]
9881 [d_loss: 0.645180, d_acc:0.71875] [g_loss: 0.760935, g_acc:0.40625]
9882 [d_loss: 0.669286, d_acc:0.625] [g_loss: 0.753998, g_acc:0.5]
9883 [d_loss: 0.698059, d_acc:0.53125] [g_loss: 0.831116, g_acc:0.3125]
9884 [d_loss: 0.691851, d_acc:0.5625] [g_loss: 0.809133, g_acc:0.21875]
9885 [d_loss: 0.726888, d_acc:0.5] [g_loss: 0.905764, g_acc:0.125]
9886 [

9996 [d_loss: 0.752375, d_acc:0.5] [g_loss: 0.877114, g_acc:0.3125]
9997 [d_loss: 0.760510, d_acc:0.5] [g_loss: 0.679254, g_acc:0.625]
9998 [d_loss: 0.617921, d_acc:0.6875] [g_loss: 0.737327, g_acc:0.4375]
9999 [d_loss: 0.661841, d_acc:0.59375] [g_loss: 0.863876, g_acc:0.25]


### [演習]
* 中間層のノードの数を変更し、その影響を確認してみましょう
* 出力間隔(sample_interval)を小刻みにして、生成される画像の変化の様子を細かく確認してみましょう
* 畳み込み層やプーリング層を取り入れて計算し、生成画像を比較してみましょう